In [4]:
import sys
import os
import cv2
from tqdm import tqdm
sys.path.append(os.path.join(os.getcwd(), 'TransCD'))
sys.path.append(os.path.join(os.getcwd(), 'Motion-U-Net'))
sys.path.append(os.path.join(os.getcwd(), 'Motion-U-Net\\Src'))
sys.path.append(os.path.join(os.getcwd(), '..'))
from TransCD.test import main as ExtractMaskTransCD
from Src.ExtractMaskMUNet1 import get_predicted_mask as ExtractMaskMUNet1
from common_utils.choose_dataset_and_video import get_random_choice
from common_utils.get_moving_object_detections import get_contour_detections, draw_contours
from common_utils.make_video import merge_images, add_text, make_video

notebook_id = 'DL'

### Read the data

We use randomly the CD2014 or SBMnet dataset and we randomly choose a video from the chosen dataset

In [2]:
dataset_path, chosen_dataset, random_category, random_sub_category = get_random_choice(notebook_id)
print('Chosen dataset:', chosen_dataset)
print('Chosen category:', random_category)
print('Chosen sub_category:', random_sub_category)
    
class AttrDict:
    def __init__(self, **entries):
        self.__dict__.update(entries)

# Define here the model, the dataset and the category
# 1st model: https://github.com/wangle53/TransCD/tree/main
opt = AttrDict(
    net_cfg='SViT_E1_D1_32',
    train_cfg=chosen_dataset,
    save_changemap=True,
    category=random_category,
    sub_category=random_sub_category
)

Chosen dataset: CDNet_2014
Chosen category: badWeather
Chosen sub_category: skating


## **Get Predicted Motion Masks from TRANSCD Model**
In this step we get the thresholded image masks. This image mask will give us relative locations of all moving targets.

In [3]:
ExtractMaskTransCD(opt)

Testing Images:   0%|          | 3/3899 [00:04<1:46:16,  1.64s/it]


KeyboardInterrupt: 

## **Make Resulting Video**

In [ ]:
# Folder paths
original_frames_path = os.path.join(dataset_path, f'{opt.category}\\{opt.sub_category}\\input')
predicted_gt_path = os.path.join(os.getcwd(), f'TransCD\\{opt.train_cfg}\\outputs\\changemap\\{opt.sub_category}')

# Get lists of all files in the folders
original_frames = os.listdir(original_frames_path)
predicted_gt = os.listdir(predicted_gt_path)

# Define a function to extract the numeric part from the gt filename
def extract_number(filename):
    return int(filename.split('_')[0])

original_frames.sort()
predicted_gt = sorted(predicted_gt, key=extract_number)

results_folder_path = os.path.join(os.getcwd(), f'..\\datasets\\results\\{opt.train_cfg}_results') 

In [ ]:
video_frames = []
# Loop over the sorted files and process pairs
for i, (f, g) in tqdm(enumerate(zip(original_frames, predicted_gt)), 
                      total=len(original_frames), desc="Processing Images"):
    # Get full paths to the images
    frame_path = os.path.join(original_frames_path, f)
    gt_path = os.path.join(predicted_gt_path, g)
    
    # Read the images using OpenCV
    frame = cv2.imread(frame_path)  # Read the first image
    gt = cv2.imread(gt_path)
    
    frame_height, frame_width = frame.shape[:2]
    gt = cv2.resize(gt, (frame_width, frame_height), interpolation=cv2.INTER_LINEAR)
    
    detections = get_contour_detections(cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY), thresh=200, distance_threshold=0)
    rgb_detections = draw_contours(frame, detections)
    
    merged_frame = add_text(merge_images(gt, rgb_detections), 'TRANSFORMER-BASED MODEL')
    # Append to list for video
    video_frames.append(merged_frame)

In [ ]:
os.makedirs(os.path.join(results_folder_path, 'Transformer Model'), exist_ok=True)
make_video(video_frames, os.path.join(results_folder_path, 'Transformer Model', f'{opt.sub_category}.mp4'))

## **Get Predicted Motion Masks from Motion-U-Net Model**
In this step we get the thresholded image masks. This image mask will give us relative locations of all moving targets.

In [5]:
ExtractMaskMUNet1(dataset_path, chosen_dataset, os.path.join(random_category, random_sub_category))

Processing Frames:   0%|          | 0/3900 [00:02<?, ?it/s]


KeyboardInterrupt: 

## **Make Resulting Video**

In [ ]:
# Folder paths
original_frames_path = os.path.join(dataset_path, f'{opt.category}\\{opt.sub_category}\\input')
predicted_gt_path = os.path.join(
    os.getcwd(), 
    f'Motion-U-Net\\Src\\outputMaskMUNet1\\{chosen_dataset}\\{random_category}\\{random_sub_category}'
    )

# Get lists of all files in the folders
original_frames = os.listdir(original_frames_path)
predicted_gt = os.listdir(predicted_gt_path)
original_frames.sort()
predicted_gt.sort()

results_folder_path = os.path.join(os.getcwd(), f'..\\datasets\\results\\{chosen_dataset}_results') 

In [ ]:
video_frames = []
# Loop over the sorted files and process pairs
for i, (f, g) in tqdm(enumerate(zip(original_frames, predicted_gt)), 
                      total=len(original_frames), desc="Processing Images"):
    # Get full paths to the images
    frame_path = os.path.join(original_frames_path, f)
    gt_path = os.path.join(predicted_gt_path, g)
    
    # Read the images using OpenCV
    frame = cv2.imread(frame_path)  # Read the first image
    gt = cv2.imread(gt_path)
    
    frame_height, frame_width = frame.shape[:2]
    gt = cv2.resize(gt, (frame_width, frame_height), interpolation=cv2.INTER_LINEAR)
    
    detections = get_contour_detections(cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY), thresh=200, distance_threshold=0)
    rgb_detections = draw_contours(frame, detections)
    
    merged_frame = add_text(merge_images(gt, rgb_detections), 'MOTION-U-NET MODEL')
    # Append to list for video
    video_frames.append(merged_frame)

In [ ]:
os.makedirs(os.path.join(results_folder_path, 'Motion-U-Net Model'), exist_ok=True)
make_video(video_frames, os.path.join(results_folder_path, 'Motion-U-Net Model', f'{random_sub_category}.mp4'))